<h1 align='center'>Introdução à Computação Visual - 2022/1</h1>
<h3 align='center'>Trabalho Prático 1: Compressão de Imagens</h3>

Autor: Flávio Marcilio de Oliveira

## Summary

- [0. Preparação](#0-preparação)
  - [Bibliotecas Necessárias](#bibliotecas-necessárias)
  - [Funções Auxiliares](#funções-auxiliares)
  - [Matriz de Quantização](#matriz-de-quantização-utilizada)
  - [Parâmetros Necessários para Codificação e Decodificação](#parâmetros-necessários-para-codificação-e-decodificação)
- [Parte I - Codificação](#parte-i---codificação)
  - [I.1 Leitura da Imagem a ser Comprimida](#i1-leitura-da-imagem-a-ser-comprimida)
  - [I.2 Cálculo da Transformada e Quantização](#i2-cálculo-da-transformada-e-quantização)
  - [I.3 Codificação de Símbolos](#i3-codificação-de-símbolos)
    - [I.3.1 Cálculo do histograma](#i31-cálculo-do-histograma)
    - [I.3.2 Codificação utilizando o algoritmo de Huffman](#i32-codificação-utilizando-o-algoritmo-de-huffman)
    - [I.3.3 Métricas da codificação](#i33-métricas-da-codificação)
  - [I.4 Gravando a Imagem Codificada](#i4-gravando-a-imagem-codificada)
- [Parte II - Decodificação](#parte-ii---decodificação)
  - [II.1 Lendo a Imagem Codificada](#ii1-lendo-a-imagem-codificada)
  - [II.2 Calculando a Matriz Quantizada e a Transformada Inversa](#ii2-calculando-a-matriz-quantizada-e-a-transformada-inversa)
  - [II.3 Métricas de Qualidade](#ii3-métricas-de-qualidade)

## 0. Preparação


**Esta etapa deve ser executada antes das duas partes I. Codificação e II. Decodificação.**

### Bibliotecas Necessárias

In [ ]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import fftpack

### Funções Auxiliares

In [ ]:
# Calcula o histograma de uma matriz
def calcHist(mat):
    # Transformando a matriz em um vetor
    arr = mat.ravel()
    # Calculando o Histograma
    histo = {}
    for i in arr:
        chave = str(np.round(i))
        if (histo.keys().__contains__(chave)):
            histo[chave] += 1
        else:
            histo[chave] = 1
    # Calculando as probabilidades de cada símbolo
    a = 0
    for x in histo.values():
        a += x
    for k, v in histo.items():
        histo[k] = v/a
    return histo

# Calcula a entropia da imagem
def calcEntropy(img):
    hist = calcHist(img)
    values =  np.array(list(hist.values()))
    logs = np.log2(values + 0.00001)
    entropy = -1 * (values*logs).sum()
    return entropy

# Algoritmo de Huffman
from heapq import heappush, heappop, heapify
from collections import defaultdict

def encode(symb2freq):
    """Huffman encode the given dict mapping symbols to weights"""
    heap = [[wt, [sym, ""]] for sym, wt in symb2freq.items()]
    heapify(heap)
    while len(heap) > 1:
        lo = heappop(heap)
        hi = heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    return sorted(heappop(heap)[1:], key=lambda p: (len(p[-1]), p))

# Configuração da apresentação da imagem
def mostra_img_unica(img, titulo='Imagem'):
    plt.imshow(img, cmap='gray', clim=(0,255))
    plt.title(titulo)
    plt.axis('off')
    plt.show()

# Calcula a DCT de uma imagem
def get_2D_dct(img):
    return fftpack.dct(fftpack.dct(img.T, norm='ortho').T, norm='ortho')

# Calcula a DCT inversa
def get_2D_idct(coefficients):
    return fftpack.idct(fftpack.idct(coefficients.T, norm='ortho').T, norm='ortho')

# Calcula a raiz do erro médio quadrático
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

# Calcula a relação sinal-ruído de pico
def psnr(predictions, targets):
    return 20 * np.log10(255/rmse(predictions, targets))

### Matriz de Quantização Utilizada

In [ ]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

# Matriz de quantização
QM = np.matrix([[16, 11, 10, 16, 24, 40, 51, 61], 
                [12, 12, 14, 19, 26, 58, 60, 55],
                [14, 13, 16, 24, 40, 57, 69, 56],
                [14, 17, 22, 29, 51, 87, 80, 62],
                [18, 22, 37, 56, 68, 109, 103, 77],
                [24, 35, 55, 64, 81, 104, 113, 92],
                [49, 64, 78, 87, 103, 121, 120, 101],
                [72, 92, 95, 98, 112, 100, 103, 99]])

QM = QM.astype(float)

print("\nMatriz de quantização")
print(QM)

### Parâmetros Necessários para Codificação e Decodificação

In [ ]:
# Imagem a ser comprimida
imagem = 'vista.jpg'

# Imagem comprimida sempre definida com extensão txt
imagem_comprimida = 'vista.txt'

# Tamanho padrão do bloco para execução da transformada
step = 8

# Parte I - Codificação

## I.1 Leitura da Imagem a ser Comprimida

In [ ]:
# Lendo imagem original em escala de cinza
img_original = cv2.imread(imagem, cv2.IMREAD_GRAYSCALE)
mostra_img_unica(img_original, titulo='Imagem a ser comprimida')

## I.2 Cálculo da Transformada e Quantização

In [ ]:
# Visualização das imagens na forma de matriz
img_original = img_original.astype(float)
print("\nImagem original:")
print(img_original)

img_transladada = img_original - 128
print("\nImagem subtraida de 128:")
print(img_transladada)

# Dimensões da imagem lida
altura, largura = img_original.shape

DCT = np.zeros((altura, largura))
matriz_quantizada = np.zeros((altura, largura))

for linha in range(0, altura, step):
    for coluna in range(0, largura, step):
        if ((img_transladada[linha:linha+step,coluna:coluna+step]).shape == (step,step)):
            dct = get_2D_dct(img_transladada[linha:linha+step, coluna:coluna+step])
            DCT[linha:linha+step, coluna:coluna+step] = dct
            matriz_quantizada[linha:linha+step, coluna:coluna+step] = np.divide(dct, QM)

print("\nMatriz quantizada:")
print(matriz_quantizada)

mostra_img_unica(DCT, "Imagem da DCT")

## I.3 Codificação de Símbolos
### I.3.1 Cálculo do Histograma

In [ ]:
histo = calcHist(matriz_quantizada)

### I.3.2 Codificação utilizando o algoritmo de Huffman

In [ ]:
# Codificação
huff = encode(histo)

print ("Symbol\tWeight\tHuffman Code")
for p in huff:
    if (histo[p[0]] > 0):
        print ("%s\t%f\t%s" % (p[0], histo[p[0]], p[1]))

### I.3.3 Métricas da Codificação

In [ ]:
nbits = 0
for i in range(0, len(huff)):
    nbits += len(huff[i][1]) * histo[huff[i][0]] * altura * largura
    
# bits per pixel
bpp = nbits / (altura * largura)
entimg1 = calcEntropy(img_original)
print ("\nSummary:")    
print ("Image entropy: %5.3f" % entimg1)
print ("size: %d bytes" % (nbits / 8))
print ("%5.3f bits/pixel" % bpp)
print ("code efficiency: %5.3f" % (entimg1/bpp))
print ("Compression ratio (without header): %5.3f" % (8/bpp))

## I.4 Gravando a Imagem Codificada

In [ ]:
arq = open(imagem_comprimida, 'w')

# Gravando o cabeçalho - Dicionário da Codificação Huffman
for i in range(0, len(huff)):
    arq.write("%s %s" %(huff[i][0], huff[i][1]))
    arq.write("\n")

# Utilizado para indicar fim do cabeçalho
arq.write("\n")

# Gravando a imagem codificada
for linha in range(0, altura):
    for coluna in range(0, largura):
        chave = str(np.round(matriz_quantizada[linha][coluna]))
        arq.write("%s" %dict(huff)[chave])
    arq.write("\n")
arq.close()

# Parte II - Decodificação

## II.1 Lendo a Imagem Codificada

In [ ]:
# Recriando o dicionário com a codificação Huffman
read = open(imagem_comprimida, 'r')
linha_lida = read.readline()
cod_huff = {}
while (linha_lida != '\n'):
    dicionario = linha_lida.split(' ')
    cod_huff[dicionario[1].removesuffix('\n')] = dicionario[0]
    linha_lida = read.readline()


# Fazendo a decodificação e recriando a matriz quantizada
bit_lido = read.read(1)
matriz_quantizada_lida = []
linha_matriz = []
linha_corrente = 0
coluna_corrente = 0
chave = ''

while True:
    while (bit_lido != '\n'):
        chave += bit_lido
        if(cod_huff.keys().__contains__(chave)):
            linha_matriz.append(float(cod_huff[chave]))
            coluna_corrente += 1
            chave = ''
        bit_lido = read.read(1)
    linha_corrente += 1
    matriz_quantizada_lida.append(linha_matriz)
    linha_matriz = []
    coluna_corrente = 0
    bit_lido = read.read(1)
    if not bit_lido:
        break

read.close()
matriz_quantizada_lida = np.matrix(matriz_quantizada_lida)

In [ ]:
# Código Huffman lido
print ("Symbol\tHuffman Code")
for k, v in cod_huff.items():
    print ("%s\t%s" % (v, k))

In [ ]:
# Matriz quantizada lida
print("\nMatriz quantizada lida:")
print(matriz_quantizada_lida)

## II.2 Calculando a Matriz Quantizada e a Transformada Inversa

In [ ]:
# Fazendo a dequantização e calculando a Transformada Inversa
altura, largura = matriz_quantizada_lida.shape
IDCT = np.zeros((altura, largura))
matriz_dequantizada = np.zeros((altura, largura))

for linha in range(0, altura, step):
    for coluna in range(0, largura, step):
        if ((matriz_quantizada_lida[linha:linha+step,coluna:coluna+step]).shape == (step,step)):
            idct = np.multiply(matriz_quantizada_lida[linha:linha+step,coluna:coluna+step], QM)
            matriz_dequantizada[linha:linha+step, coluna:coluna+step] = idct
            idct = get_2D_idct(idct)
            IDCT[linha:linha+step, coluna:coluna+step] = idct

# Somando 128 na matriz da transformada inversa
img_decodificada = IDCT + 128

print("\nMatriz Dequantizada:")
print(matriz_dequantizada)

print("\nIDCT da imagem:")
print(IDCT)

mostra_img_unica(img_decodificada, "Imagem reconstruida")

## II.3 Métricas de Qualidade

In [ ]:
img_original = cv2.imread(imagem, cv2.IMREAD_GRAYSCALE)
print ("RMSE: %5.3f" % rmse(img_original, img_decodificada))
print ("PSNR: %5.3f" % psnr(img_original, img_decodificada))